In [0]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer
from pyspark.sql.functions import udf, col, lower, regexp_replace
import re
import pandas as pd
import numpy as np
import string




Out[21]: DataFrame[Insult: int, Comment: string]

In [0]:
 
import re
import string
from pyspark.sql.functions import udf, col, lower, regexp_replace


df = spark.read.csv('/FileStore/tables/cyberbullydata.csv',inferSchema=True,header=True)
spark.conf.set("spark.sql.legacy.timeParserPolicy","Legacy")

df=df.select(['Insult','Comment'])

df.dropna()


def preProcess(iter):
    
        
        # remove extra space
        regex_ws=re.compile("\s+")
        ret=regex_ws.sub(" ",iter)
        ret=ret.replace("&amp;","&").replace("&lt;","<").replace("&gt;",">").replace("\\x","")
        #text="".join([word for word in text if word not in string.punctuation])
        
        #Replace URL
        regexp="(https?:\/\/(?:www\.|(?!www)|(?:xmlns\.))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})"
        ret=re.sub(regexp,"url",ret)
        url_pattern = re.compile(r'https?://\S+|www\.\S+')
        ret= url_pattern.sub(r'', ret)
  
        #replace @addresses
        regexp='@[A-z0-9_]+'
        ret=re.sub(regexp,"@refertoperson",ret)

        #Bring everything to lowercase
        ret=ret.lower()
 
        #removing punctuations 
        #ret2="".join([char for char in ret if char not in string.punctuation])

        return ret2
    
preProcudf=udf(preProcess)
df = df.withColumn('Comment', preProcudf('Comment'))
display(df.selectExpr("Insult", "Comment"))
        

Insult,Comment
0,the drudge reportnnnnyou wont see this story on foxfag forum because they suck biacks and gay 0bama all the way to the cracknnnnnnnnon tuesday rep darrell issa
0,refertopersonroger clemens is the fucking man
1,agree with alan you are an extremest idiot you are not american
0,reallynni see marc lamont hill on various programs quite often
0,really suck isnt the word
0,after a couple guys like him rape him first
0,matthews mentored perry at usc bodes well
0,how about we accept facts and patiently wait for you to back up your claim
1,to engage in an intelligent debate with you is like debating to a retarded person its useless it looks like youre bent on disregarding the efforts of the government
0,same reason no one cared obama did cocaine


In [0]:
from sparknlp.annotator import DocumentNormalizer, Tokenizer, LemmatizerModel, StopWordsCleaner, Normalizer
from sparknlp.base import DocumentAssembler
from pyspark.ml import Pipeline

# Creates document formats
documentAssembler = (DocumentAssembler()\
                     .setInputCol('Comment')\
                     .setOutputCol('Document')
                    )
doc_norm_patterns = ["<[^>]>","_","https?:\/\/.*[\r\n]*", '"', r"\\n", r"\\u\d+","0_0"]

# Cleans Documents
documentNormalizer = DocumentNormalizer() \
    .setInputCols("Document") \
    .setOutputCol("NormalizedDocument") \
    .setAction("clean") \
    .setPatterns(doc_norm_patterns) \
    .setReplacement(" ") \
    .setPolicy("pretty_all") \
    .setLowercase(True)

# Tokenizes
tokenizer = Tokenizer()\
    .setInputCols(["NormalizedDocument"])\
    .setOutputCol("Token").fit(df)
# Normalizer - cleans tokens remaining
normalizer = Normalizer() \
    .setInputCols(["Token"]) \
    .setOutputCol("Normalized") \
    .setCleanupPatterns([r"\\x.\d",r"\\","@[^\s]+"])

# # Spell Checker
# spellChecker = ContextSpellCheckerModel \
#     .pretrained() \
#     .setTradeoff(12.0) \
#     .setInputCols("Token") \
#     .setOutputCol("Checked")


# Lemmatize
lemmatizer = LemmatizerModel.pretrained() \
    .setInputCols(["Normalized"]) \
    .setOutputCol("Lemma")

# Remove Stop Words
stopWords = StopWordsCleaner() \
     .setInputCols(["Lemma"]) \
     .setOutputCol("CleanTokens") \
     .setCaseSensitive(False)

pipeline = Pipeline().setStages([
    documentAssembler,
    documentNormalizer,
    tokenizer,
    normalizer,
    lemmatizer,
    stopWords
])

#sample_data = spark.createDataFrame(df)
# data = spark.createDataFrame([[text]]).toDF("Text")
pipelineModel = pipeline.fit(df)
result = pipelineModel.transform(df)
display(result.selectExpr("Insult", "Comment","CleanTokens.result"))


lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[ | ][OK!]

Insult,Comment,result
0,the drudge reportnnnnyou wont see this story on foxfag forum because they suck biacks and gay 0bama all the way to the cracknnnnnnnnon tuesday rep darrell issa,"List(drudge, reportnnnnyou, wont, see, story, foxfag, forum, suck, biacks, gay, 0bama, way, cracknnnnnnnnon, tuesday, rep, darrell, issa)"
0,refertopersonroger clemens is the fucking man,"List(refertopersonroger, clemens, fucking, man)"
1,agree with alan you are an extremest idiot you are not american,"List(agree, alan, extreme, idiot, american)"
0,reallynni see marc lamont hill on various programs quite often,"List(reallynni, see, marc, lamont, hill, various, program, quite, often)"
0,really suck isnt the word,"List(really, suck, isnt, word)"
0,after a couple guys like him rape him first,"List(couple, guy, like, rape, first)"
0,matthews mentored perry at usc bodes well,"List(matthew, mentore, perry, usc, bode, well)"
0,how about we accept facts and patiently wait for you to back up your claim,"List(accept, fact, patiently, wait, back, claim)"
1,to engage in an intelligent debate with you is like debating to a retarded person its useless it looks like youre bent on disregarding the efforts of the government,"List(engage, intelligent, debate, like, debate, retarded, person, useless, look, like, youre, bend, disregard, effort, government)"
0,same reason no one cared obama did cocaine,"List(reason, one, care, obama, cocaine)"


In [0]:


train_data,test_data=df.randomSplit([0.7,0.3])

In [0]:

pip install nltk
# Import stemmer library
import nltk
from nltk.stem.porter import *

# Instantiate stemmer object
stemmer = PorterStemmer(inputCol='MeaningfulWords',
                       outputCol='OrgWords')



print(stemmer.stem(t))

In [0]:
#tokenize
tokenizer=Tokenizer(inputCol='Comment',outputCol='Commentwords')
tok_train=tokenizer.transform(df)
display(tok_train.selectExpr("Insult", "Comment","Commentwords"))


Insult,Comment,Commentwords
0,the drudge reportnnnnyou wont see this story on foxfag forum because they suck biacks and gay 0bama all the way to the cracknnnnnnnnon tuesday rep darrell issa,"List(the, drudge, reportnnnnyou, wont, see, this, story, on, foxfag, forum, because, they, suck, biacks, and, gay, 0bama, all, the, way, to, the, cracknnnnnnnnon, tuesday, rep, darrell, issa)"
0,refertopersonroger clemens is the fucking man,"List(refertopersonroger, clemens, is, the, fucking, man)"
1,agree with alan you are an extremest idiot you are not american,"List(agree, with, alan, you, are, an, extremest, idiot, you, are, not, american)"
0,reallynni see marc lamont hill on various programs quite often,"List(reallynni, see, marc, lamont, hill, on, various, programs, quite, often)"
0,really suck isnt the word,"List(really, suck, isnt, the, word)"
0,after a couple guys like him rape him first,"List(after, a, couple, guys, like, him, rape, him, first)"
0,matthews mentored perry at usc bodes well,"List(matthews, mentored, perry, at, usc, bodes, well)"
0,how about we accept facts and patiently wait for you to back up your claim,"List(how, about, we, accept, facts, and, patiently, wait, for, you, to, back, up, your, claim)"
1,to engage in an intelligent debate with you is like debating to a retarded person its useless it looks like youre bent on disregarding the efforts of the government,"List(to, engage, in, an, intelligent, debate, with, you, is, like, debating, to, a, retarded, person, its, useless, it, looks, like, youre, bent, on, disregarding, the, efforts, of, the, government)"
0,same reason no one cared obama did cocaine,"List(same, reason, no, one, cared, obama, did, cocaine)"
